In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp as pl
pd.set_option('display.max_rows', 200000)

In [2]:
carbon_emissions_df  = pd.read_csv('carbon_emissions.csv')
resale_df = pd.read_csv('cost_profiles.csv')
demand_df  = pd.read_csv('demand.csv')
fuels_df = pd.read_csv('fuels.csv')
vehicles_df  = pd.read_csv('vehicles.csv')
vehicles_fuels_df  = pd.read_csv('vehicles_fuels.csv')
sample_submission_df = pd.read_csv('sample_submission.csv') 

### Edit vehicles_df to include all distance bucket selections

In [3]:
# Define a function to generate additional rows based on 'Distance'
def generate_rows(row):
    if row['Distance'] == 'D4':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D3'}]),
            pd.DataFrame([{**row, 'Distance': 'D2'}]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    elif row['Distance'] == 'D3':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D2'}]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    elif row['Distance'] == 'D2':
        return pd.concat([
            pd.DataFrame([row]),
            pd.DataFrame([{**row, 'Distance': 'D1'}])
        ])
    else:
        return pd.DataFrame([row])

# Apply the function to each row and concatenate the results
vehicles_df_add = pd.concat(vehicles_df.apply(generate_rows, axis=1).tolist(), ignore_index=True)

In [4]:
vehicles_df_add

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2026,BEV,S1,2026,160329,102000,D1
5,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
6,BEV_S1_2027,BEV,S1,2027,152312,102000,D1
7,BEV_S1_2028,BEV,S1,2028,144697,102000,D2
8,BEV_S1_2028,BEV,S1,2028,144697,102000,D1
9,BEV_S1_2029,BEV,S1,2029,137462,102000,D3


### Initialize the Pulp problem and define decision variables

Unfortunatly, we will assume the consumption will be maximised for selected vehicle for now.

In [5]:
# Initialize the PuLP problem
prob = pl.LpProblem("Fleet_Transition_Optimization", pl.LpMinimize)

# Define Fuel type
fuel_type=['Electricity', 'B20', 'LNG', 'BioLNG', 'HVO']

# Define Distance Bucket type
distance_bucket = ['D1', 'D2', 'D3', 'D4']

# Define decision variables
buy_vars = pl.LpVariable.dicts("Buy", 
                               ((year, vehicle_id) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']),
                               lowBound=0, cat='Integer')

use_vars = pl.LpVariable.dicts("Use", 
                               ((year, vehicle_id, fuel, db) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']
                               for fuel in fuel_type for db in distance_bucket),
                               lowBound=0, cat='Integer')

sell_vars = pl.LpVariable.dicts("Sell", 
                                ((year, vehicle_id) for year in range(2023, 2039) for vehicle_id in vehicles_df['ID']),
                                lowBound=0, cat='Integer')

### Define objective function

The total cost consists of three components:
- Buying Cost for the vehicle
- Using Cost for the vehicle
- (Negative/Gain) Resale cost

Where the Using cost consists of three components:
- Fuel cost
- Insurance Cost
- Maintenance Cost

The Fuel cost is calculated by:

`D_v_f`: Distance travelled by vehicle `v` using fuel `f`. **(Assumed max)**
Unit: km

**multiplies by**

`N_v_f`: Number of vehicles of type `v` driving fuel type `f`.

**multiplies by**

`m_v_f`: Fuel consumption of vehicle of type `v` driving with fuel type `f`.
Unit: unit_fuel/km

**multiplies by**

`C_f_yr`: Cost of unit fuel of type `f` in the year `yr`.
Unit: $/unit_fuel

In [6]:
# First calculate buying costs
total_cost = pl.lpSum([
    # Buying cost
    buy_vars[(year, row['ID'])] * row['Cost ($)']
    for year in range(2023, 2039)
    for _, row in vehicles_df.iterrows()
])

In [7]:
# Add insurance and maintenance costs
for year in range(2023, 2039):
    for v in vehicles_df['ID']:
        purchase_year = int(v.split('_')[-1])
        if year >= purchase_year:
            age = year - purchase_year+1
            if age < 10:
                # Normal vehicle price
                purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v, 'Cost ($)'].values[0]
                
                # Insurance percentage by age
                insurance_cost_pct = resale_df.loc[resale_df['End of Year'] == age, 'Insurance Cost %'].values[0] / 100
                
                # Maintenance percentage by age
                maintenance_cost_pct = resale_df.loc[resale_df['End of Year'] == age, 'Maintenance Cost %'].values[0] / 100
                
                # Insurance cost 
                insurance_cost = purchase_cost * insurance_cost_pct
                
                # Maintenance cost
                maintenance_cost = purchase_cost * maintenance_cost_pct
                
                # Add insurance cost to total cost
                total_cost += insurance_cost * pl.lpSum([use_vars[(year, v, fuel, db)] for db in distance_bucket for fuel in fuel_type])
                
                # Add maintenance cost to total cost
                total_cost += maintenance_cost * pl.lpSum([use_vars[(year, v, fuel, db)] for db in distance_bucket for fuel in fuel_type])

Before we calculate, we will merge the dataframe vehicles_df, vehicles_fuels_df, and fuels_df.

First, merge fuels_df with vehicles_df_add using column `Year`

In [8]:
merged_vehicles_df = pd.merge(vehicles_fuels_df, vehicles_df_add, on='ID', how='outer')

In [9]:
merged_fuels_total_df = pd.merge(merged_vehicles_df, fuels_df, on=['Fuel', 'Year'], how = 'outer')

In [10]:
# Add fuel costs
for year in range(2023, 2039):
    for db in distance_bucket:
        for fuel in fuel_type:
            for _, row in merged_fuels_total_df.iterrows():
                # Unique vehicle id
                vehicle_id = row['ID']
                
                # Max distance travelled by vehicle, unit: km
                distance = row['Yearly range (km)']
                
                # Fuel type
                fuel_t = row['Fuel']
                
                # Fuel cost, unit: $/unit
                fuel_cost = row['Cost ($/unit_fuel)']
                
                # Fuel consumption, unit: unit/km
                fuel_consumption = row['Consumption (unit_fuel/km)']
                
                # Append calculated fuel cost to total cost
                ## Formula: Fuel_cost = number of vehicle v * Distance * Fule_Consumption * Fuel_cost
                total_cost += distance * fuel_cost * fuel_consumption * use_vars[(year, vehicle_id, fuel, db)]

In [11]:
# Add (Negative/Gain) Resale cost
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        purchase_year = int(vehicle_id.split('_')[-1])
        if year > purchase_year:
            age = year - purchase_year + 1
            if age < 10:
                purchase_cost = merged_fuels_total_df.loc[merged_fuels_total_df['ID'] == vehicle_id, 'Cost ($)'].values[0]
                resale_value_pct = resale_df.loc[resale_df['End of Year'] == age, 'Resale Value %'].values[0] / 100
                resale_value = purchase_cost * resale_value_pct
                total_cost -= resale_value * sell_vars[(year, vehicle_id)]

In [12]:
prob += total_cost

***

### Setting Constraints

In [13]:
# 1. Ensure vehicles can only be bought in the specific year
for year in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        purchase_year = int(v.split('_')[-1])
        if year != purchase_year:
            prob += buy_vars[(year, v)] == 0

In [14]:
# 2. Ensure vehicle has a 10-year life and must be sold by the end of 10th year, and also must buy, then sell.
for v in merged_fuels_total_df['ID']:
    purchase_year = int(v.split('_')[-1])
    if purchase_year+10 <2040:
        prob += buy_vars[(purchase_year, v)] == pl.lpSum([sell_vars[(y, v)] for y in range(purchase_year, purchase_year+10)])
    else:
        prob += buy_vars[(purchase_year, v)] >= pl.lpSum([sell_vars[(y, v)] for y in range(purchase_year, 2039)])

In [15]:
# 3. Ensure vehicles are used only if they have been bought
for v in merged_fuels_total_df['ID']:
    for db in distance_bucket:
        for fuel in fuel_type:
            purchase_year = int(v.split('_')[-1])

            prob += buy_vars[(purchase_year, v)] >= pl.lpSum([use_vars[(y, v, fuel, db)] for y in range(purchase_year, 2039)])

            if purchase_year != 2023:
                for y in range(2023, purchase_year):
                    prob += use_vars[(y, v, fuel, db)] == 0

In [16]:
# 4. Every year at most 20% of the vehicles in the existing fleet can be sold
for v in merged_fuels_total_df['ID']:
    if int(v.split('_')[-1]) == 2023:
        prob += sell_vars[(2023, v)]<= 0.2 * buy_vars[(2023, v)]

for year in range(2024, 2039):
    for v in merged_fuels_total_df['ID']:
        prob += sell_vars[(year, v)] <= 0.2 * (pl.lpSum([buy_vars[(y, v)] for y in range(2023, year+1)]) - pl.lpSum([sell_vars[y, v] for y in range(2023, year)]))

In [17]:
# 5. Yearly demand must be met for each distance bucket
for year in range(2023, 2039):
    for db in ['D1', 'D2', 'D3', 'D4']:
        for size in ['S1', 'S2', 'S3', 'S4']:
            demand = demand_df[(demand_df['Year'] == year) & (demand_df['Distance'] == db) & (demand_df['Size'] == size)]['Demand (km)'].values[0]
            prob += pl.lpSum([use_vars[(year, v, fuel, db)] * vehicles_df_add.loc[vehicles_df_add['ID'] == v , 'Yearly range (km)'].values[0]
                           for v in vehicles_df_add['ID'] if (vehicles_df_add.loc[vehicles_df_add['ID'] == v, 'Distance'].values[0] >= db) & (vehicles_df_add.loc[vehicles_df_add['ID'] == v, 'Size'].values[0] == size)]) >= demand

In [18]:
# 6. Carbon emission limits must be respected
for year in range(2023, 2039):
    for db in distance_bucket:
        for fuel in fuel_type:
            for _, row in merged_fuels_total_df.iterrows():
                # Unique vehicle id
                vehicle_id = row['ID']
                
                # Max distance travelled by vehicle, unit: km
                distance = row['Yearly range (km)']
                
                # Fuel type
                fuel_t = row['Fuel']
                
                # Carbon emissions, unit: (CO2/unit_fuel)
                emissions = row['Emissions (CO2/unit_fuel)']
                
                # Fuel consumption, unit: unit/km
                fuel_consumption = row['Consumption (unit_fuel/km)']
                
                # Calculate total emissions
                total_emissions = pl.lpSum([use_vars[(year, vehicle_id, fuel, db)] * fuel_consumption * distance * emissions])
    prob += total_emissions <= carbon_emissions_df[carbon_emissions_df['Year'] == year]['Carbon emission CO2/kg'].values[0]

In [21]:
# Ensure all vehicles can only use selected fuel type
for year in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        vt = v.split('_')[0]
        for ft in fuel_type:
            for db in distance_bucket:
                if ((vt == 'BEV') & (ft != 'Electricity')) | ((vt == 'Diesel') & (ft not in ['B20', 'HVO'])) | ((vt == 'LNG') & (ft not in ['LNG', 'BioLNG'])):
                    prob += use_vars[(year, v, ft, db)] == 0  

***

In [22]:
prob.solve()

-1

In [26]:
for y in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        for d in distance_bucket:
            for f in fuel_type:
                if pl.value(buy_vars[(y, v)]) >0:
                    print(y,v, pl.value(buy_vars[(y, v)]))

2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diesel_S1_2030 116.08829
2030 Diese

In [24]:
for y in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        for d in distance_bucket:
            for f in fuel_type:
                if pl.value(use_vars[(y, v, f, d)]) >0:
                    print((y,v,f, d, pl.value(use_vars[(y, v, f, d)]) ))

(2023, 'Diesel_S1_2023', 'HVO', 'D1', 2.1303456)
(2023, 'Diesel_S1_2023', 'HVO', 'D2', 6.3654265)
(2023, 'Diesel_S1_2023', 'HVO', 'D3', 8.0686544)
(2023, 'Diesel_S1_2023', 'HVO', 'D4', 1.0154779)
(2023, 'Diesel_S1_2023', 'HVO', 'D1', 2.1303456)
(2023, 'Diesel_S1_2023', 'HVO', 'D2', 6.3654265)
(2023, 'Diesel_S1_2023', 'HVO', 'D3', 8.0686544)
(2023, 'Diesel_S1_2023', 'HVO', 'D4', 1.0154779)
(2023, 'Diesel_S1_2023', 'HVO', 'D1', 2.1303456)
(2023, 'Diesel_S1_2023', 'HVO', 'D2', 6.3654265)
(2023, 'Diesel_S1_2023', 'HVO', 'D3', 8.0686544)
(2023, 'Diesel_S1_2023', 'HVO', 'D4', 1.0154779)
(2023, 'Diesel_S1_2023', 'HVO', 'D1', 2.1303456)
(2023, 'Diesel_S1_2023', 'HVO', 'D2', 6.3654265)
(2023, 'Diesel_S1_2023', 'HVO', 'D3', 8.0686544)
(2023, 'Diesel_S1_2023', 'HVO', 'D4', 1.0154779)
(2023, 'Diesel_S2_2023', 'HVO', 'D1', 2.3483349)
(2023, 'Diesel_S2_2023', 'HVO', 'D2', 3.2622547)
(2023, 'Diesel_S2_2023', 'HVO', 'D3', 1.8349245)
(2023, 'Diesel_S2_2023', 'HVO', 'D4', 0.31527594)
(2023, 'Diesel_S2_2

In [27]:
for y in range(2023, 2039):
    for v in merged_fuels_total_df['ID']:
        for d in distance_bucket:
            for f in fuel_type:
                if pl.value(sell_vars[(y, v)]) >0:
                    print(y, v, pl.value(sell_vars[(y, v)]))

2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diesel_S1_2030 23.217657
2031 Diese

In [20]:
output = []
for year in range(2023, 2039):
    for vehicle_id in vehicles_df['ID']:
        if pl.value(buy_vars[(year, vehicle_id)]) > 0:
            output.append([
                year,
                vehicle_id,
                pl.value(buy_vars[(year, vehicle_id)]),
                'Buy',
                'NaN',
                'NaN',
                0.0
            ])
        if pl.value(use_vars[(year, vehicle_id, fuel, db)]) > 0:
            for db in ['D1', 'D2', 'D3', 'D4']:
                if pl.value(distance_bucket_choice_vars[(year, vehicle_id, distance_bucket)]) > 0:
                    for fuel in ['Electricity', 'B20', 'LNG', 'BioLNG', 'HVO']:
                        if pl.value(fuel_choice_vars[(year, vehicle_id, fuel)]) > 0:
                            output.append([
                                year,
                                vehicle_id,
                                pl.value(use_vars[(year, vehicle_id)]),
                                'Use',
                                fuel,
                                distance_bucket,
                                pl.value(distance_vars[(year, vehicle_id)])
                            ])
        if pl.value(sell_vars[(year, vehicle_id)]) > 0:
            output.append([
                year,
                vehicle_id,
                pl.value(sell_vars[(year, vehicle_id)]),
                'Sell',
                'NaN',
                'NaN',
                0.0
            ])

output_df = pd.DataFrame(output, columns=[
    'Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)'
])

NameError: name 'distance_bucket_choice_vars' is not defined